## GBP for CisDecoding batch
### outputing heatmap, weight raw data, weight integrated among genes/channels

このファイルは保存用です。
ファイルをコピーし、ファイル名を変更した後、使用する。

使用にはutilフォルダが必要です。同一フォルダ内にutilフォルダをコピーして使用する。

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from visualizations_forCisDecode import GradCAM, GuidedGradCAM, GBP, LRP, CLRP, SGLRP, LRPA, LRPB, LRPE
from keras.layers import (Activation, Add, GlobalAveragePooling2D,
                          BatchNormalization, Conv1D, Conv2D, Dense, Flatten, Reshape, Input, Dropout,
                          MaxPooling1D,MaxPooling2D)

from keras.models import load_model, Model, Sequential
from helper_forCisDecode import heatmap_optional
import innvestigate.utils as iutils
import os
from keras.utils import plot_model,np_utils
from keras.preprocessing.image import img_to_array, load_img

from keras.layers import Dense, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from keras.regularizers import l2
from keras import backend as K
from keras import optimizers

from functools import reduce
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from sklearn.metrics import classification_report, confusion_matrix

import skimage as sk
sk.__version__

# limits tensorflow to a specific GPU
os.environ["CUDA_VISIBLE_DEVICES"]="0"

Using TensorFlow backend.
/home/deepstation/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/deepstation/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/deepstation/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([(

In [2]:
input_shape=(82055,20,50,1)### k=50に設定。(XXX, length, channel, 1): 1000-bpのとき
num_classes=2

from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
    model = load_model("./Bin25_MGBRup1.2_lr1e-4_ep10_ROC0.610.h5")

# model.compile(loss='categorical_crossentropy', optimizer='Nadam',metrics=['accuracy'])###lr設定必要
print(model.summary())  #modelのsummaryを表示

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 20, 32)            4832      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 20, 32)            3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 10, 32)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 32)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 10, 32)            3104      
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 10, 32)            3104      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 5, 32)             0         
__________

In [6]:
# Only the partial model is needed for the visualizers(GradCam用). 
# Use innvestigate.utils.keras.graph.pre_softmax_tensors()
# dense名が変更するまで、何回かRunさせる必要がある。
partial_model = Model(
    inputs=model.inputs,
    outputs=iutils.keras.graph.pre_softmax_tensors(model.outputs),
    name=model.name,
)
partial_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1_input (InputLayer)  (None, 20, 50)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 20, 32)            4832      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 20, 32)            3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 10, 32)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 32)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 10, 32)            3104      
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 10, 32)            3104      
__________

In [7]:
# Range of input images
# keras_applications VGG16 weights assume a range of (-127.5, 127.5). 
# Change this to a range suitable for your model.

max_input = 1
min_input = 0

# max_input = -127.5
# min_input = 127.5

In [ ]:
### train内の全遺伝子リストに対してpredictionを算出 ###

import glob
pred_list = open("Prediction-list.txt", "w")### この名前でprediction listが保存される

files = glob.glob("./gene_dataset/inputs/train*/*.npy") ### /gene_dataset/inputs/trainXX/内の.npyファイル名を取得
for file in files:
    input_img = np.load(file)
    input_img_1 = np.reshape(input_img,(1,20,50))
    input_img_2 = np.reshape(input_img,(20,50))

    pred_score = model.predict(input_img_1)
    print(str(os.path.split(sfile)[1]) + " prediction score:" + pred_score + "\n", file=pred_list)

pred_list.close()


In [8]:
### 特定のnpyファイルのみ(現在は./selected/)に関してGBPをかけてheatmap or rawData output ###
import glob

imagefolder = "./GBPheatmap-cisdecode/" ### heatmap画像保存フォルダを指定
rawfolder = "./GBPrawdata-cisdecode/"   ### raw dataの保存フォルダを指定
#
### GBPの関数呼び出し ###
guidedbackprop_analyzer = GBP(
    partial_model,
    target_id=1,### 基本的にはpositive (target id =1) のものを対象にする
    relu=True,
)


### 画像の色閾値と透明度の設定 ###
rate = 0.5 ### 最大値のr倍が白色に: つまり0.5なら0.5がmin-1.0がmax
alpha = 0.5 ### 透明度 (alpha値)の設定。0 (無色) - 1 (透かしなし)
transp = "False" ### 背景を透明にする場合はTrue、しない場合はFalse


sfiles = glob.glob("./selected/*.npy") ### /selected/内の.npyファイル名を取得
for sfile in sfiles:
    ### heatmap保存 ###
    fig = plt.figure(figsize=(20, 8), dpi=600)
    input_img = np.load(sfile)
    input_img_1 = np.reshape(input_img,(1,20,50))
    analysis_guidedbackprop = guidedbackprop_analyzer.analyze(input_img_1)
    heatmap_optional(analysis_guidedbackprop[0], r=rate, alp=alpha) ### 最大値のr倍が白色に: つまり0.5なら0.5がmin-1.0がmax alpha値も。
#     plt.show()
    fig.savefig(imagefolder + str(os.path.split(sfile)[1]) + "_r=" + str(rate) + ".png", transparent=transp) ### ファイル名_r=XX.png という名前で上で指定した画像フォルダに保存

    ### raw data保存 ###
    datafile = open(rawfolder + str(os.path.split(sfile)[1]), "w")
    print(str(analysis_guidedbackprop), file=datafile)

In [9]:
### GBPは遺伝子間での値が標準化されていないので、スケール感が異なる。したがって遺伝子間での足し算はNGである（2020/11/21に気づいた)

### おまけ①。selectedにある遺伝子のGBP weightを「遺伝子間」で統合して「TFweights_Intg_selectedGenes.txt」に保存 ###

# intg = open("TFweights_Intg_selectedGene.txt","w") ### この名前のファイルに保存
# weight = np.zeros((1,20,50),dtype=float)
# for sfile in sfiles:
#     input_img = np.load(sfile)
#     input_img_1 = np.reshape(input_img,(1,20,50))
#     analysis_guidedbackprop = guidedbackprop_analyzer.analyze(input_img_1)
#     weight = analysis_guidedbackprop + weight
# print(str(weight), file=intg)
# intg.close()

In [28]:
### おまけ②。selectedにある遺伝子のGBP weightを「TFチャンネル」に統合して「TFweights_Intg_TFchannels.txt」に保存 ###

intg2 = open("TFweights_Intg_TFchannel.txt","w") ### この名前のファイルに保存
rates = 0.5 ### heatmap optionに合わせて、最大値のrates倍以下を切る（min値の設定)

for sfile in sfiles:
    input_img = np.load(sfile)
    input_img_1 = np.reshape(input_img,(1,20,50))
    analysis_guidedbackprop = guidedbackprop_analyzer.analyze(input_img_1)
    gbp = np.reshape(analysis_guidedbackprop,(20,50))

    weight2 = np.zeros(50,dtype=float)
    threshold = max(gbp.reshape(1000)) * rates
    for i in range(len(gbp)):
        gbp_relative = gbp[i] - threshold ### アレイ中最大値のrates倍をminとする（ただし、マイナス値も取るので、一行下の関数）
        gbp_clip = np.clip(gbp_relative,0,max(gbp.reshape(1000))) ### 閾値の設定、min=0, max=アレイ中の最大値。
        weight2 = gbp_clip + weight2
    print(str(os.path.split(sfile)[1])+ ": " + str(weight2) + "\n", file = intg2)

intg2.close()


In [11]:
### おまけ③ TFリストの順番
glob.glob("./raw_datas/*.txt") ### 「今raw_datas」に入っているものなので注意。

['./raw_datas/k50_BinSum2Binary25_wOTU-HSFA6B_colamp_a.h5-TomatoAllprom1k-count0.8_Float2Binary.txt',
 './raw_datas/k50_BinSum2Binary25_wOTU-AtIDD11_colamp_a.h5-TomatoAllprom1k-count0.8_Float2Binary.txt',
 './raw_datas/k50_BinSum2Binary25_wOTU-IDD7_col_a.h5-TomatoAllprom1k-count0.8_Float2Binary.txt',
 './raw_datas/k50_BinSum2Binary25_wOTU-At5g47390_col_b.h5-TomatoAllprom1k-count0.8_Float2Binary.txt',
 './raw_datas/k50_BinSum2Binary25_wOTU-At5g58900_colamp_a.h5-TomatoAllprom1k-count0.8_Float2Binary.txt',
 './raw_datas/k50_BinSum2Binary25_wOTU-CUC3_col_a.h5-TomatoAllprom1k-count0.8_Float2Binary.txt',
 './raw_datas/k50_BinSum2Binary25_wOTU-ANAC005_col_a.h5-TomatoAllprom1k-count0.8_Float2Binary.txt',
 './raw_datas/k50_BinSum2Binary25_wOTU-MYB70_col_a.h5-TomatoAllprom1k-count0.8_Float2Binary.txt',
 './raw_datas/k50_BinSum2Binary25_wOTU-ANAC050_col_v3a.h5-TomatoAllprom1k-count0.8_Float2Binary.txt',
 './raw_datas/k50_BinSum2Binary25_wOTU-MYB99_colamp_a.h5-TomatoAllprom1k-count0.8_Float2Binary